<a href="https://colab.research.google.com/github/rcorcoranadams/GeoPandas-and-Folium-Tutorial/blob/main/tutorial_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Using GeoPandas, MatPlot and Folium in Tandem: A Geospatial Exploration using Retrograde Problem Solving***

**Rachel Corcoran-Adams**

Date: December 7, 2020

Python 2.7.5

Inputs: Shapefiles obtained through MassGIS and the American Community Survey 

Outputs: Projected, clipped, selected and then visualized geojson files using GeoPandas, MatPlot and Folium

In [ ]:
!pip install geopandas
!pip install gdal
!pip install rtree
!apt-get install -y libspatialindex-dev
!install -g topojson
import gdal
import matplotlib.pyplot as plt
import rtree
import folium
import os
import json
from folium import plugins
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt

In [ ]:
# import files from Drive
from google.colab import drive
drive.mount('/content/gdrive')
# set root path
root_path = 'gdrive/My Drive/GIS/final_project/'

Mounted at /content/gdrive


In [ ]:
# import all of the shapefiles 
open_space = gpd.read_file(root_path+'open_space_clipped_1.shp')
ACS_Worcester = gpd.read_file(root_path+'ACS_Clipped_final_1.shp')
census_tracts = gpd.read_file(root_path+'CENSUS2010TRACTS_POLY.shp')
census_towns = gpd.read_file(root_path+ 'CENSUS2010TOWNS_POLY.shp')
landcover = gpd.read_file(root_path+ 'landcover_clipped.shp')
vehicle_access = gpd.read_file(root_path+ 'vehicle_access.shp')

In [ ]:
# project all the shapefiles 
open_space = open_space.to_crs("EPSG:4326")
ACS_Worcester = open_space.to_crs("EPSG:4326")
census_tracts = census_tracts.to_crs("EPSG:4326")
census_towns = census_towns.to_crs("EPSG:4326")
landcover = landcover.to_crs("EPSG:4326")
vehicle_access = vehicle_access.to_crs("EPSG:4326")

In [ ]:
#check to see if it worked!
open_space.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#check to see if it worked!
ACS_Worcester.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#check to see if it worked!
census_tracts.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#check to see if it worked!
landcover.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#check to see if it worked!
vehicle_access.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# This line selects by attribute to create a worcester polygon
worcester = census_towns[census_towns['TOWN']=="WORCESTER"]
# This line clips the open space file to just include worcester
open_space_clipped = gpd.clip(open_space, worcester)
# This line selects by attribute for open space to include public access
open_space_select = open_space_clipped[open_space_clipped['PUB_ACCESS']=="Y"]
# This line selects only polygons with full public access and with primary purpose of recreation and conservation
pub_access_recreation = open_space_select[open_space_select['PRIM_PURP']== "B"] 
# This line will be selecting only the residential multi-family features in landcover 
landcover_residential = landcover[landcover['USEGENCODE']== "Residential-multi-family"]

In [ ]:
# Creates a function to convert the shapefiles into geojson 
def shapefile2geojson(infile, outfile, fieldname):
    options = gdal.VectorTranslateOptions(format="GeoJSON",
                                          dstSRS="EPSG:4326")
    gdal.VectorTranslate(outfile, infile, options=options)
# returns the function with the files as parameters 
shapefile2geojson('pub_access_recreation.geojson', 'pub_access_recreation.shp', 'SITE_NAME')
shapefile2geojson('landcover_residential.geojson', 'landcover_residential.shp', 'COVERNAME')
shapefile2geojson('worcester.geojson', 'worcester.shp', 'TOWN')
shapefile2geojson('vehicle_access.geojson', 'vehicle_access.shp', 'B08201_002E')

In [ ]:
# Visualize the layer that shows locations of public green space in Worcester using matplot
pub_access_recreation.plot(column='SITE_NAME', color='green', figsize=(16,8));

In [ ]:
# Visualize the layer that shows households with no vehicle access in Worcester using matplot
vehicle_access.plot(column='TARGET_FID', color='red', figsize=(16,8));

In [ ]:
def write_json(self, features):
   # feature is a shapely geometry type
   geom_in_geojson = geojson.Feature(geometry=features, properties={})
   tmp_file = tempfile.mkstemp(suffix='.geojson')
   geojson.dump(geom_in_geojson, 'pub_access_recreation.geojson')
   pub_access_recreation.to_file(root_path+'pub_access_recreation.geojson') 

In [ ]:
# Shows the vehicle access Geojson
folium.GeoJson(
    vehicle_access
).add_to(Vehicle_map)
Vehicle_map